<center>
<img src="../../img/ods_stickers.jpg">
## Открытый курс по машинному обучению
<center>Автор: [Дмитрий Дрёмов](https://www.kaggle.com/dremovd)
    


## Простой бенчмарк в соревновании по категоризации покупок

[Kaggle Inclass](https://www.kaggle.com/c/receipt-categorisation) и отдельная [ссылка](https://www.kaggle.com/t/73f1a2eb0be9443ba1f8d2f283adc444) для участия. Недавно на чеках появились QR коды. Пока еще не все с этим знакомы, но по информации из этого кода можно получить полное содержание чека. Это дает возможность вести расходы, учитывая каждый отдельный товар, включая расходы, сделанные наличными. Как следствие наличия полной информации, можно анализировать изменения характера расходов и инфляцию по собственной продуктовой корзине. Названия товаров не стандартизованы: у одного товара в разных магазинах существенно отличаются названия; отдельные слова могут сокращаться; названия могут содержать опечатки. В магазины постоянно добавляются новые товары. Это делает простое составление каталога всех товаров с категориями нереалистичным.

Данные публикуются впервые, а обученные на них модели используются в [production](https://play.google.com/store/apps/details?id=com.dremovd.fnschecks&hl=ru). Задача, которую предлагается решить — это разбиение всех покупок чека по небольшому набору понятных человеку категорий.

In [1]:
import os
import pandas as pd
import numpy as np

# Загрузка данных

In [2]:
PATH_TO_DATA = 'data'
RANDOM_STATE = 17

In [3]:
!ls $PATH_TO_DATA

apteka.csv.gz	 catalog2.csv.gz  remont.csv.gz       train.csv.gz
cafe.csv.gz	 catalog3.csv.gz  test.csv.gz	      train_checks.csv.gz
catalog1.csv.gz  deti.csv.gz	  test_checks.csv.gz


In [4]:
train = pd.read_csv(os.path.join(PATH_TO_DATA, 'train.csv.gz'))
train_checks = pd.read_csv(os.path.join(PATH_TO_DATA, 'train_checks.csv.gz'))

In [5]:
print(train.shape, train_checks.shape)
print(train.columns.values)
print(train_checks.columns.values)

(13682, 5) (2042, 4)
['check_id' 'name' 'category' 'price' 'count']
['check_id' 'shop_name' 'datetime' 'sum']


In [6]:
train.fillna('', inplace=True)

# Подготовка данных

In [7]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()

In [8]:
X = vectorizer.fit_transform(train.name)

In [9]:
for i, word in enumerate(vectorizer.vocabulary_):
    print(word)
    if i > 10:
        break

смороди
кампомос
penosill
специальная
kлубниkа
заглушка
пилзнер
капус
bc
пальмы
рзпс
яйцом


In [10]:
X.shape

(13682, 15758)

In [11]:
from sklearn.preprocessing import LabelEncoder

labeler = LabelEncoder()

In [12]:
y = labeler.fit_transform(train.category)

# Валидация

In [13]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GroupShuffleSplit

gkf = list(GroupShuffleSplit(n_splits=5, random_state=RANDOM_STATE).split(X, y, train.check_id.values))
score = cross_val_score(LogisticRegression(random_state=RANDOM_STATE), X, y, scoring='neg_log_loss',
                        cv=gkf, n_jobs=-1)
"%.3f +- %.4f" % (-np.mean(score), np.std(score))

'0.745 +- 0.0164'

# Подготовка сабмита

In [14]:
model = LogisticRegression(random_state=RANDOM_STATE)
model.fit(X, y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=17, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [15]:
test = pd.read_csv(os.path.join(PATH_TO_DATA, 'test.csv.gz'))
test_checks = pd.read_csv(os.path.join(PATH_TO_DATA, 'test_checks.csv.gz'))

In [16]:
X_test = vectorizer.transform(test.name)

In [17]:
p_test = model.predict_proba(X_test)

In [18]:
test = test[['id']]

In [19]:
for i, c in enumerate(labeler.classes_):
    test[c] = list(map(lambda x: '%.3f' % x, p_test[:, i]))

In [21]:
test.to_csv('base_solution.csv.gz', compression='gzip', index = False, encoding='utf-8')

У такой посылки результат – 0.65081 в [публичном рейтинге](https://www.kaggle.com/c/receipt-categorisation/leaderboard) соревнования, бенчмарк "Стартер".